In [22]:
#Libraries for data processing
import pandas as pd
import numpy as np


In [23]:
from logging import RootLogger
# Mount Google Drive
from google.colab import drive # import drive from google colab

root = "/content/drive"        # default location for the drive

drive.mount(root)              # we mount the google drive at /content/drive

# import join used to join root path and my_google_drive_path
from os.path import join  

# path to your project on Google Drive
my_google_drive_path = "MyDrive/StudentProject2023"

project_path = join(root, my_google_drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###We will calculate some more data for DZ10
-Binding Energy

(-S1p & S2n)

-S2p & S2n

Data is already in MeV

#### A and Binding Energy

In [24]:
#Reading DZ10 data as binding_energy
dz_data = pd.read_csv(join(project_path,"processed_data/dz_data.csv"), sep=";")

#We create a new column in pandas dataframe "dz_data" containing mass number
dz_data["A"] = dz_data["Z"] + dz_data["N"]

#We create a new column containing Binding Energy not divided by A
dz_data["dz_BE"] = dz_data["dz_BE/A"] * dz_data["A"]

#I think this line is useless
#dz_data.sort_values(by="A", ascending=True)

#### Computation of S1n and S1p


In [25]:
dz_data.sort_values(by=['Z'], ascending=True)
dz_data['dz_S1n'] = dz_data['dz_BE'] - dz_data['dz_BE'].shift(1)


dz_data = dz_data.sort_values(by=['N','A'], ascending=True)
dz_data['dz_S1p'] = dz_data['dz_BE'] - dz_data['dz_BE'].shift(1)

#### Computation of S2n and S2p

In [26]:
#Data already well sorted by the computation of S1p
#dz_data = dz_data.sort_values(by=['N','A'], ascending=True)
dz_data['dz_S2p'] = dz_data['dz_BE'] - dz_data['dz_BE'].shift(2)

dz_data = dz_data.sort_values(by=['Z','A'], ascending=True)
dz_data['dz_S2n'] = dz_data['dz_BE'] - dz_data['dz_BE'].shift(2)

### We will calculate some data for AME

-Binding energy 

-S2n & S2p

We will also drop some data that we finally don't use

#### Binding Energy

In [27]:
ame_data = pd.read_csv(join(project_path,"processed_data/mass_data.csv"), sep=";")

ame_data['ame_BE'] = ame_data['ame_BE/A'] * ame_data['A']

#### Computation of S2n and S2p

In [28]:
ame_data.sort_values(by=['Z'], ascending=True)
ame_data['ame_S2p'] = ame_data['ame_BE'] - ame_data['ame_BE'].shift(2)

ame_data = ame_data.sort_values(by=['Z','A'], ascending=True)
ame_data['ame_S2n'] = ame_data['ame_BE'] - ame_data['ame_BE'].shift(2)

### From these two documents we will merge all the data we need in one dataframe and csv and also create the training and validation datasets

#### First of all, we will change units and calculate some quantities that may be useful to train our artificial neural networks

In [29]:
merged_data=pd.merge(dz_data, ame_data, on=["Z", "N", "A"])

#Change of units
#AME file gives energies in keV
#DZ10 file gives energies in MeV
#We want MeV at the end
merged_data["ame_BE/A"] = merged_data["ame_BE/A"]/1000
merged_data["ame_ME"] = merged_data["ame_ME"]/1000
merged_data["ame_BE"] = merged_data["ame_BE"]/1000
merged_data["ame_S2p"] = merged_data["ame_S2p"]/1000
merged_data["ame_S2n"] = merged_data["ame_S2n"]/1000

#We compute the difference in binding energy AME - DZ10
merged_data["BE_diff_dz_ame"] = merged_data["ame_BE/A"]*merged_data["A"] - merged_data["dz_BE"]


#### We compute some quantities related to the Liquid Drop Model

In [30]:
alphaV=15.49 
alphaS=17.23
alphaC=0.697
alphaA=23.285
alphaP=12

merged_data["Surf"] = np.power(merged_data["A"],2/3)
merged_data["Asym"] = ( (merged_data["N"]-merged_data["Z"])**2 ) / merged_data["A"]
merged_data["Coul"] = ( merged_data["Z"]*(merged_data["Z"]-1) ) / np.power(merged_data["A"],1/3)
merged_data["Pair"] = np.power(merged_data["A"],-1/2)
merged_data["Z_parity"] = np.power(-1,merged_data["Z"])
merged_data["N_parity"] = np.power(-1,merged_data["N"])

merged_data["ldm_BE"]=alphaV * merged_data["A"] - alphaS * merged_data["Surf"] - alphaC * merged_data["Coul"] - alphaA * merged_data["Asym"]

#### Then we compute others quantities that give more physics terms such as the parity for N and Z, and their distance with respect to magic numbers

In [31]:
merged_data["Z_parity"] = np.power(-1,merged_data["Z"])
merged_data["N_parity"] = np.power(-1,merged_data["N"])

magic_numbers = [2, 8, 20, 28, 50, 82, 126, 184]

#Adding two columns for the distance with respect to magic numbers
merged_data["Z_distance"] = None
merged_data["N_distance"] = None

#Compute the distance to magic numbers
for i, row in merged_data.iterrows():
    z = row["Z"]
    n = row["N"]
    merged_data.at[i, "Z_distance"] = min([abs(z - m) for m in magic_numbers])
    merged_data.at[i, "N_distance"] = min([abs(n - m) for m in magic_numbers])

In [32]:
#We save this merged dataframe to .csv
merged_data.to_csv(join(project_path,"processed_data/merged_data.csv"),sep=";", index=False)

In [33]:

#From the merged table, create one training dataset and a validation one
train_data = pd.DataFrame(columns=["Z","N","dz_BE/A","dz_ME","A","dz_BE","dz_S1n","dz_S1p","dz_S2p", "dz_S2n","ame_ME", "ame_BE/A", "ame_AM", "ame_BE", "ame_S2p", "ame_S2n", "BE_diff_dz_ame","Surf","Asym","Coul","Pair","Z_parity","N_parity","Z_distance","N_distance"])
validation_data = pd.DataFrame(columns=["Z","N","dz_BE/A","dz_ME","A","dz_BE","dz_S1n","dz_S1p","dz_S2p", "dz_S2n","ame_ME", "ame_BE/A", "ame_AM", "ame_BE", "ame_S2p", "ame_S2n", "BE_diff_dz_ame","Surf","Asym","Coul","Pair","Z_parity","N_parity","Z_distance","N_distance"])


#We separate the merged dataframe into training and validation datasets
for i in range(len(merged_data)) :
    
    if int(merged_data.iloc[i]["Z"]) in [10,38,54,68,82] :
        validation_data = validation_data.append(merged_data.iloc[i], ignore_index=True)

    else :
        train_data = train_data.append(merged_data.iloc[i], ignore_index=True)


#We don't use data before A<16 because these light nuclei experience
#At least for training
#Physics effects that are very far from trivial (halo etc)
train_data.drop(train_data[(train_data["A"]<16 )].index, inplace=True)

train_merged_csv = train_data.to_csv(join(project_path,"processed_data/train_merged_data.csv"),sep=";")
validation_merged_csv = validation_data.to_csv(join(project_path,"processed_data/validation_merged_data.csv"),sep=";")